In [4]:
#libraries

from termcolor import colored

In [5]:
def readfile(file):
    with open(file,'r') as f:
        content=f.read()
    return content

In [8]:
content=readfile('texto.txt')
print(colored('To start with your English practice, read the text below:\n','blue'))
print(content)

To start with your English practice, read the text below:

What if thatâ€™s not actually true? Perhaps itâ€™s more useful to consider that in every moment, on every project, no one is actually doing their best.

Because thereâ€™s always a need to hold a little bit in reserve.

Because there are always competing priorities.

Because everyone has a noise in their head.

Because thereâ€™s fear, a hundred kinds of fear.

Because no one has actually done the lopsided work of 100% preparation and commitment, not for this precise moment.

Iâ€™m not doing my best and neither are you. Because weâ€™re not optimized algorithms, weâ€™re people.

Okay, now that we can see that no one is doing their best, what are our options?


In [9]:
def cleaning(file):
    #replace punctuation marks
    with open(file,'r') as f:
        content=f.read().lower().replace('.','').replace('-','').replace(',','').replace('  ',' ').replace('?','').replace('!','').replace('"','').replace('’','').split()
        
    #drop small words
    small_words=[x for x in content if len(x) <=3]
    for word in small_words:
        if word in content:
            content.remove(word)

    #unique words
    a_set=set(content)
    a_list=list(a_set)
    return a_list

In [10]:
original_content=cleaning('texto.txt')
#print(original_content)

In [11]:
def interaction0():
    user_words=input("Write the words that you remember: ")
    split_user_words=user_words.split()
    return split_user_words

In [12]:
user_words=interaction0()

Write the words that you remember: people their best your option kind better what moment


In [13]:
#print(user_words)

In [14]:
true_w=[]
false_w=[]
def evaluation(original_content,user_words):
    for word in user_words:
        if word in original_content:
            true_w.append(word)
        else:
            false_w.append(word)
    return true_w, false_w

In [15]:
evaluation(original_content,user_words)

(['people', 'their', 'best', 'what', 'moment'],
 ['your', 'option', 'kind', 'better'])

In [16]:
#print(true_w, false_w)

In [17]:
def user_performance(original_content,true_w,false_w):
    percent=int((len(true_w)/len(original_content))*100)
    if percent >= 50:
        print('Success rate:',percent,'%','\nTerrific! You have good memory.')
    elif percent <= 50:
        print('Success rate:',percent,'%','\nUhhhmm You can do it better... Try again!')
    print(colored("Words contained in the text:",'green'),true_w)
    print(colored("Words not contained in the text:",'red'),false_w)

In [18]:
user_performance(original_content,true_w,false_w)

Success rate: 10 % 
Uhhhmm You can do it better... Try again!
Words contained in the text: ['people', 'their', 'best', 'what', 'moment']
Words not contained in the text: ['your', 'option', 'kind', 'better']


In [19]:
def interaction1():
    user_words1=input("Do you want to know more about a specific word?: ")
    split_user_words1=user_words1.split()
    return split_user_words1

In [20]:
more_info=interaction1()

Do you want to know more about a specific word?: kind


# Obtaining definitions, examples ...

In [21]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup as bs

C:\Users\Sativa\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


#### 1. From the VERB FORMS (csv of web scraping)

In [22]:
df_verbs = pd.read_csv('verb_forms.csv') 
#df_verbs.head()

In [23]:
#df_verbs.columns

In [24]:
df_verbs.drop(['Unnamed: 0'], axis=1, inplace=True)
#df_verbs.head()

In [25]:
#mejorar esto porque no pude pasar las palabras de word_info u_u

lamask=np.column_stack([df_verbs[col].str.contains(r'\bborn\b') for col in df_verbs])
df_verbs.loc[lamask.any(axis=1)]

,base_form,past_form,past_participle,s_es_ies,ing_form
48,bear,bore,born,bears,bearing


#### 2. From the DICTIONARY (json from Kaggle)

In [26]:
with open('dictionary.json','r') as mydict:
    dictionary=mydict.read()

In [27]:
datadict=json.loads(dictionary)
#datadict

In [28]:
#testing
#datadict['before']

In [29]:
for w in more_info:
    if w in datadict:
        print(colored(w,'green'),datadict[w])
    else:
        print(colored(w,'red'),'not found in dictionary!')

kind ['Possessing charm and attractiveness.', 'A category of things distinguished by a common characteristic or quality.']


#### 3. From  API Oxford

In [30]:
app_id = '7f3799d9'
app_key = '07d636d3ecf38d32b2ffbf299a13e871'
language = 'en'
#word_id = 'effort'
#url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()

In [31]:
def busqueda(word_id):
    definition=[]
    url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()
    
    #request 
    r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
    #print("code {}\n".format(r.status_code))
    
    #json
    a_dict = r.json()
    a_dict
    
    #busqueda dentro del json para la definicion
    definition.append(a_dict['results'][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions'])
    
    return definition 

In [35]:
for word in more_info: #more_info es la lista de palabras que el usuario quiere saber más
    try:
        definition=busqueda(word)
        print(colored(word,'green'),definition)
    except:
        print(colored(word,'red'),'Not found! or Key error')

kind [['a group of people or things having similar characteristics']]


#### 4. From web scraping (lexico-translation)

In [36]:
def translation(word):
    translations=[]
    url='https://www.lexico.com/en-es/translate/'+word
    html=requests.get(url).content
    soup=bs(html,'lxml')
    trnsltns=soup.find_all("span", {"class": "hw head-translation"})
    translations.append(trnsltns)
    return translations

In [37]:
for word in more_info: #more_info es la lista de palabras que el usuario quiere saber más
    try:
        w_trans=translation(word)
        print(colored(word,'green'),w_trans)
    except:
        print(colored(word,'red'),'Not found!')

kind [[<span class="hw head-translation">tipo, <em class="rs_skip">n.</em></span>, <span class="hw head-translation">amable, <em class="rs_skip">adj.</em></span>]]
